In [1]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.block = {}
        self.start = nn.Sequential(
            nn.Conv2d(3,64,7,2,3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3,2,1)
        )
        
        self.a1_1,self.b1_1 = self.Residual(64,64,3,1)
        self.a1_2,self.b1_2 = self.Residual(64,64,3,1)
        self.a1_3,self.b1_3 = self.Residual(64,64,3,1)
        
        self.a2_1,self.b2_1 = self.Residual(64,128,3,2)
        self.a2_2,self.b2_2 = self.Residual(128,128,3,1)
        self.a2_3,self.b2_3 = self.Residual(128,128,3,1)
        self.a2_4,self.b2_4 = self.Residual(128,128,3,1)
        
        self.a3_1,self.b3_1 = self.Residual(128,256,3,2)
        self.a3_2,self.b3_2 = self.Residual(256,256,3,1)
        self.a3_3,self.b3_3 = self.Residual(256,256,3,1)
        self.a3_4,self.b3_4 = self.Residual(256,256,3,1)
        self.a3_5,self.b3_5 = self.Residual(256,256,3,1)
        self.a3_6,self.b3_6 = self.Residual(256,256,3,1)
        
        self.a4_1,self.b4_1 = self.Residual(256,512,3,2)
        self.a4_2,self.b4_2 = self.Residual(512,512,3,1)
        self.a4_3,self.b4_3 = self.Residual(512,512,3,1)
        
        self.relu = nn.ReLU(inplace=True)
        self.avg = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(512,2)
        
    def forward(self,x):
        x = self.start(x)
        
        x = self.Block(self.a1_1,self.b1_1,x)
        x = self.Block(self.a1_2,self.b1_2,x)
        x = self.Block(self.a1_3,self.b1_3,x)
        
        x = self.Block(self.a2_1,self.b2_1,x)
        x = self.Block(self.a2_2,self.b2_2,x)
        x = self.Block(self.a2_3,self.b2_3,x)
        x = self.Block(self.a2_4,self.b2_4,x)
        
        x = self.Block(self.a3_1,self.b3_1,x)
        x = self.Block(self.a3_2,self.b3_2,x)
        x = self.Block(self.a3_3,self.b3_3,x)
        x = self.Block(self.a3_4,self.b3_4,x)
        x = self.Block(self.a3_5,self.b3_5,x)
        x = self.Block(self.a3_6,self.b3_6,x)
        
        x = self.Block(self.a4_1,self.b4_1,x)
        x = self.Block(self.a4_2,self.b4_2,x)
        x = self.Block(self.a4_3,self.b4_3,x)
        
        x = self.avg(x)
        
        x = self.linear(x.view(x.size()[0],-1))
        
        return x
    
    def Residual(self,C_in,C_out,size,stride):
            over = nn.Sequential(
                    nn.Conv2d(C_in,C_out,size,stride,1),
                    nn.BatchNorm2d(C_out),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(C_out,C_out,size,1,1),
                    nn.BatchNorm2d(C_out)
                )
            if C_in !=C_out:   
                change = nn.Sequential(
                        nn.Conv2d(C_in,C_out,1,stride),
                        nn.BatchNorm2d(C_out)
                    )
            else:
                change = nn.Sequential()
            return over,change
        
    def Block(self,a,b,x):
        out = a(x)
        x = b(x)
        x = self.relu(x+out)
        return x
    
net = Net().cuda()
print(net)
net.load_state_dict(torch.load('params_res_train_2_15.pkl'))
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
loss_func=nn.CrossEntropyLoss()

Net(
  (start): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (a1_1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (b1_1): Sequential()
  (a1_2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [2]:
import h5py
def load_dataset():
    train_dataset=h5py.File('train_res_dataset.h5','r')
    train_x=train_dataset['train_x'][:]
    train_y=train_dataset['train_y'][:]
    train_x=torch.FloatTensor(train_x)
    train_y=torch.LongTensor(train_y)
    return train_x,train_y
train_x,train_y=load_dataset()
print(train_x.size(),train_y.size(),train_y)

torch.Size([25000, 3, 64, 64]) torch.Size([25000]) tensor([0, 0, 0,  ..., 1, 1, 1])


In [3]:
import torch.utils.data as Data
train_dataset=Data.TensorDataset(train_x,train_y)
train_loader=Data.DataLoader(
    dataset=train_dataset,
    batch_size=512,
    shuffle=True,
    pin_memory=True,
    num_workers=4
)

In [4]:
for epoch in range(200):
    for step,(b_x,b_y) in enumerate(train_loader):
        b_x=b_x.cuda()
        b_y=b_y.cuda()
        output=net(b_x)
        loss=loss_func(output,b_y).cuda()
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step()
    scheduler.step(loss)
    if epoch % 10 == 0:
        print('第'+str(epoch)+'次成本:'+str(loss))                 

第0次成本:tensor(0.0001, device='cuda:0', grad_fn=<NllLossBackward>)
第10次成本:tensor(0.0008, device='cuda:0', grad_fn=<NllLossBackward>)
第20次成本:tensor(2.4359e-06, device='cuda:0', grad_fn=<NllLossBackward>)
第30次成本:tensor(3.1264e-07, device='cuda:0', grad_fn=<NllLossBackward>)
第40次成本:tensor(2.5428e-06, device='cuda:0', grad_fn=<NllLossBackward>)
第50次成本:tensor(3.7787e-07, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch    59: reducing learning rate of group 0 to 1.0000e-04.
第60次成本:tensor(3.7112e-08, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch    70: reducing learning rate of group 0 to 1.0000e-05.
第70次成本:tensor(7.3100e-08, device='cuda:0', grad_fn=<NllLossBackward>)
第80次成本:tensor(2.6428e-07, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch    89: reducing learning rate of group 0 to 1.0000e-06.
第90次成本:tensor(1.5632e-07, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch   101: reducing learning rate of group 0 to 1.0000e-07.
第100次成本:tensor(2.5866e-08, device='cuda:0', grad_fn=<NllLossB

In [5]:
torch.save(net.state_dict(),'params_res_train_2_16.pkl')
print('保存成功')

保存成功


In [6]:
torch.save(net.state_dict(),'params_res_train_2_16_two.pkl')
print('保存成功')

保存成功
